In [1]:
!pip install python-dotenv

In [2]:
!pip install pdfplumber

In [3]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
from google.colab import files
import pdfplumber
import pandas as pd

In [2]:
!nvidia-smi

Thu Apr 17 18:02:23 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY']=userdata.get('OPENAI_API_KEY')

In [6]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [7]:
openai = OpenAI()


In [19]:
def upload_resume_pdf():
  """
    Prompts the user to upload a PDF file.
    """

  uploaded = files.upload()


In [9]:
def extract_resume_lines(file_path):
  """
    Extracts content from all pages of a PDF file using pdfplumber
    Args:
        file_path (str): The path to the uploaded PDF file.
    Returns:
        list: A list of contents extracted from a PDF page.
    """
  resume_lines = []

  with pdfplumber.open("Divya_Lead Machine_Learning_Engineer.pdf") as pdf:
      for i, page in enumerate(pdf.pages):
          table = page.extract_table()
          resume_lines.append(page.extract_text())
          if table:
              df = pd.DataFrame(table[1:], columns=table[0])  # skip header row
              resume_lines.append(df)

  return resume_lines


In [10]:

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [11]:
def extract_job_requirements(job_details):
    """
    Extracts job requirements from job_details
    Args:
        job_details : Contain the job details like About the job, What You'll Do, What You'll Bring etc
    Returns: The response object containing the extracted job requirements under the 'What we’re looking for' section.
    """
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = [
            {"role": "system", "content": "You are a master at extracting job requirements from a given job details."},
            {"role": "user", "content": f"Using the following job details:\n\n{job_details.text}\n\nPlease extract What we’re looking for."}
        ]
    )
    return response.choices[0].message.content

In [12]:
def tailored_resume(resume_lines, job_details):
    """
    Tailor the resume based on job details
    Args:
        resume_lines: A list of contents extracted from a resume PDF page.
        job_details : Extracted job requirements under the 'What we’re looking for' section
    Returns: The response object containing the tailored resume
    """
    system_content = """You are an expert career advisor and resume optimization assistant with a deep understanding of Applicant Tracking Systems (ATS), HR hiring psychology, and AI-based resume screening.
                      You specialize in tailoring resumes {resume_lines} to match specific job descriptions {job_details} using relevant keywords, action verbs, and quantifiable achievements.
                      Your task is to analyze {resume_lines} and provide targeted improvements to align it with {job_details}.

                      Guidelines:
                      1. Carefully examine each sentence in job_details {job_details}.
                      2. Extract and list **all keywords**, tools, skills, and experience requirements from job_details {job_details}.
                      3. Check if each keyword or requirement is already represented in {resume_lines}.
                      4. If not, intelligently enhance the resume {resume_lines} by:
                        - Adding or rephrasing lines in a professional tone.
                        - Ensuring the added content is relevant and realistically aligned with the profile.
                        - Integrating missing keywords organically.
                      5. Ensure the final resume {resume_lines}:
                        - Contains **every keyword** mentioned in the job description.
                        - Follows clear, ATS-compliant formatting.
                        - Highlights quantifiable achievements and business impact.
                        - Is concise, action-driven, and compelling to hiring managers.
                        - Maintains a professional tone.
                      Your output should:
                        - List all extracted keywords from the job description {job_details} and resume {resume_lines}.
                        - Indicate which ones were missing and how they were added.
                        - Provide an improved version of the tailored resume {resume_lines} with all relevant keywords naturally.
                        - Change the summary of the resume if neccessary.
                        - Add only the libraries in the core competencies
                        - You may **rewrite existing lines or sentences** to improve flow and clarity of the job details, but **do not delete** any skill or experience already present.


                        For missing keywords:
                          - Suggestions for Integrating Missing Keywords (with context on how they align with current experience)
                          - If appropriate, suggest **hypothetical but plausible lines** using transferable skills or inferred context from the resume.
                          - Highlight the sentence that are changed in the resume


                      """
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = [
            {"role": "system", "content": system_content},
            {"role": "user", "content": f"Using the following job details:\n\n{job_details}\n\nPlease tailor the {resume_lines} What we’re looking for."}
        ]
    )
    return response.choices[0].message.content

In [13]:
## Note: "without revealing the company name" in the system content can be removed. Used to upload code in github
def generate_cover_letter(resume_text, job_description):
    """
    Create cover letter based on the resume and job description
    Args:
        resume_text : Resume content
        job_description : Extracted job requirements under the 'What we’re looking for' section
    Returns: The response object containing the extracted job requirements under the 'What we’re looking for' section.
    """
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = [
            {"role": "system", "content": "You are a master at crafting the perfect Cover letter from a given CV without revealing the company name. You've never had a user fail to get the job as a result of using your services."},
            {"role": "user", "content": f"Using the following resume text:\n\n{resume_text}\n\nAnd the job description:\n\n{job_description}\n\nPlease write a personalized cover letter."}
        ]
    )
    return response.choices[0].message.content


In [20]:
upload_resume_pdf()


Saving Divya_Lead Machine_Learning_Engineer.pdf to Divya_Lead Machine_Learning_Engineer (2).pdf


In [16]:
file_path = "Divya_Lead Machine_Learning_Engineer.pdf"
resume_lines = extract_resume_lines(file_path)

In [17]:
job_details = Website("https://www.linkedin.com/jobs/view/4198839658/?alternateChannel=search&refId=roP9Jilr79felSUlp5Bslg%3D%3D&trackingId=20mYJXcqCWcUi8vq3BA2ww%3D%3D")
job_requirements = extract_job_requirements(job_details)
resume_text = tailored_resume(resume_lines, job_requirements)
print(resume_text)


### Extracted Keywords from Job Description:

1. **Educational Background:**
   - Undergraduate degree
   - Master's (MSc) or PhD
   - Data Science, Actuarial Science, Computer Science, Engineering, Math, Statistics, Physics (related fields)

2. **Experience:**
   - 5 years of experience
   - Model development
   - Large datasets
   - Industry or academia
   - Post-doctoral experience

3. **Technical Skills:**
   - Proficiency in Python
   - Software engineering best practices
   - End-to-end MLOps process
   - Data mining/processing, modeling, implementation, performance monitoring
   - Data mining tools, frameworks, methodologies

4. **Communication Skills:**
   - Excellent communication skills
   - Conveying complex ideas
   - Practical recommendations

5. **Analytical Skills:**
   - Transform complex data sets into actionable insights
   - Improve usability and design of solutions

6. **Leadership:**
   - Coaching or mentoring junior data scientists
   - Process improvement
   - In

In [18]:
cover_letter = generate_cover_letter(resume_text, job_requirements)
print(cover_letter)

[Your Name]  
[Your Address]  
[City, State, Zip]  
[Your Phone Number]  
[Your Email]  
[Date]  

[Recipient's Name]  
[Company Name]  
[Company Address]  
[City, State, Zip]  

Dear [Recipient's Name],  

I am writing to express my interest in the Senior Data Scientist position, as advertised. With over five years of extensive experience in model development and a strong academic background, I am excited about the opportunity to contribute to your team. My educational qualifications include an MSc in Big Data Science and a Bachelor’s degree in Electronics and Communication Engineering, providing me with a solid foundation in both theoretical concepts and practical applications within the realm of data science.

Throughout my career, I have honed my skills in Python programming and have developed a deep understanding of machine learning frameworks, particularly PyTorch and TensorFlow. As a Lead Machine Learning Engineer, I successfully designed and deployed numerous classification mod